In [9]:
%pip install --upgrade --quiet  rank_bm25 > /dev/null

In [12]:
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM

# Configure Ollama LLM
ollama_llm = OllamaLLM(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbeddings(
    model="nomic-embed-text:latest",  # Correct parameter name is `model`
    base_url="http://localhost:11434",  # Base URL for the Ollama service
)

In [13]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS

doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]


# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

In [ ]:


doc_list_2 = [
    "You like apples",
    "You like oranges",
]

embedding = ollama_embedding
faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})


In [14]:

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [15]:
docs = ensemble_retriever.invoke("apples")
docs

[Document(metadata={'source': 1}, page_content='I like apples'),
 Document(id='937c1298-70a2-443e-9691-b066961cba1c', metadata={'source': 2}, page_content='You like apples'),
 Document(metadata={'source': 1}, page_content='Apples and oranges are fruits'),
 Document(id='0c84faee-ad8d-42cb-bf63-859a974a7439', metadata={'source': 2}, page_content='You like oranges')]

## Runtime Configuration

We can also configure the individual retrievers at runtime using [configurable fields](/docs/how_to/configure). Below we update the "top-k" parameter for the FAISS retriever specifically:

In [10]:
from langchain_core.runnables import ConfigurableField

faiss_retriever = faiss_vectorstore.as_retriever(
    search_kwargs={"k": 2}
).configurable_fields(
    search_kwargs=ConfigurableField(
        id="search_kwargs_faiss",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [16]:
config = {"configurable": {"search_kwargs_faiss": {"k": 1}}}
docs = ensemble_retriever.invoke("apples", config=config)
docs

[Document(metadata={'source': 1}, page_content='I like apples'),
 Document(id='937c1298-70a2-443e-9691-b066961cba1c', metadata={'source': 2}, page_content='You like apples'),
 Document(metadata={'source': 1}, page_content='Apples and oranges are fruits')]

Notice that this only returns one source from the FAISS retriever, because we pass in the relevant configuration at run time